# Exploración del Dataset: EDUCACION

Importación de Librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

# Configuración
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 50)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("✅ Librerías importadas correctamente")

Carga del Dataset

In [ ]:
# Definir rutas de forma robusta
import os
from pathlib import Path

# Obtener la ubicación del notebook
notebook_dir = Path.cwd()
print(f"Directorio actual: {notebook_dir}")

# Construir ruta al directorio data/raw
if 'notebooks' in str(notebook_dir):
    DATA_RAW_PATH = Path('../../data/raw')
    DICT_PATH = Path('../../data/diccionarios')
else:
    DATA_RAW_PATH = Path(r'E:\MTPE\data\raw')
    DICT_PATH = Path(r'E:\MTPE\data\diccionarios')

print(f"Ruta DATA_RAW_PATH: {DATA_RAW_PATH.resolve()}")
print(f"Directorio existe: {DATA_RAW_PATH.exists()}")

# Cargar dataset
archivo_csv = DATA_RAW_PATH / 'DataSet_EDUCACION.csv'
print(f"Archivo CSV: {archivo_csv.name}")
print(f"Archivo existe: {archivo_csv.exists()}")

if archivo_csv.exists():
    print("\n Cargando dataset (puede tardar unos segundos por el tamaño del archivo)...")
    df_educacion = pd.read_csv(archivo_csv, encoding='utf-8-sig')

    # Verificar y limpiar filas completamente vacías
    registros_iniciales = len(df_educacion)
    filas_vacias = df_educacion.isnull().all(axis=1).sum()

    if filas_vacias > 0:
        print(f"\n Se encontraron {filas_vacias:,} filas completamente vacías")
        print(f" Limpiando filas vacías...")
        df_educacion = df_educacion.dropna(how='all').reset_index(drop=True)
        print(f" Filas vacías eliminadas: {filas_vacias:,}")

    print(f"\n Dataset cargado exitosamente")
    print(f" Dimensiones finales: {df_educacion.shape[0]:,} filas x {df_educacion.shape[1]} columnas")
    print(f" Registros válidos: {len(df_educacion):,} de {registros_iniciales:,} ({len(df_educacion)/registros_iniciales*100:.2f}%)")
else:
    print(f"\n ERROR: No se encuentra el archivo en: {archivo_csv.resolve()}")
    print(f"   Por favor, verifica la ruta del archivo.")

Vista Previa - Primeras Filas

In [ ]:
print(" Primeras 15 filas del dataset:")
display(df_educacion.head(15))

Vista Previa - Últimas Filas

In [ ]:
print("Últimas 10 filas del dataset:")
display(df_educacion.tail(10))

Vista Previa - Muestra Aleatoria

In [ ]:
print(" Muestra aleatoria de 20 registros:")
display(df_educacion.sample(min(20, len(df_educacion)), random_state=42))

Información General del Dataset

In [ ]:
print("Información del Dataset:")
print("=" * 80)
df_educacion.info()

print("\n Columnas del Dataset:")
print("=" * 80)
for idx, col in enumerate(df_educacion.columns, 1):
    print(f"{idx:2d}. {col} ({df_educacion[col].dtype})")

Análisis de Valores Nulos

In [ ]:
print(" Análisis de Valores Nulos:")
print("=" * 80)

null_data = pd.DataFrame({
    'Columna': df_educacion.columns,
    'Nulos': df_educacion.isnull().sum(),
    '% Nulos': (df_educacion.isnull().sum() / len(df_educacion) * 100).round(2)
}).sort_values(by='Nulos', ascending=False)

display(null_data)

if null_data['Nulos'].sum() > 0:
    plt.figure(figsize=(12, 6))
    null_cols = null_data[null_data['Nulos'] > 0]
    plt.barh(null_cols['Columna'], null_cols['% Nulos'])
    plt.xlabel('Porcentaje de Valores Nulos (%)')
    plt.title('Valores Nulos por Columna - EDUCACION')
    plt.tight_layout()
    plt.show()
else:
    print("✅ No hay valores nulos en el dataset")

Análisis de Duplicados

In [ ]:
print(" Análisis de Duplicados:")
print("=" * 80)

duplicados_totales = df_educacion.duplicated().sum()
print(f"Registros duplicados (completos): {duplicados_totales:,} ({duplicados_totales/len(df_educacion)*100:.2f}%)")

# Buscar columna de ID
id_cols = [col for col in df_educacion.columns if 'ID' in col.upper()]
if id_cols:
    for id_col in id_cols:
        duplicados_id = df_educacion[id_col].duplicated().sum()
        print(f"Duplicados en {id_col}: {duplicados_id:,} ({duplicados_id/len(df_educacion)*100:.2f}%)")

        if duplicados_id > 0 and duplicados_id < 100:
            print(f"\n Ejemplos de {id_col} duplicados:")
            ids_duplicados = df_educacion[df_educacion[id_col].duplicated(keep=False)][id_col].value_counts().head(10)
            display(ids_duplicados)

Estadísticas Descriptivas - Variables Numéricas

In [ ]:
print(" Estadísticas Descriptivas - Variables Numéricas:")
print("=" * 80)
display(df_educacion.describe())

Estadísticas Descriptivas - Variables Categóricas

In [ ]:
print(" Estadísticas Descriptivas - Variables Categóricas:")
print("=" * 80)
display(df_educacion.describe(include=['object']))

Análisis de Cardinalidad

In [ ]:
print(" Análisis de Cardinalidad:")
print("=" * 80)

# Calcular valores únicos para cada columna
valores_unicos = [df_educacion[col].nunique() for col in df_educacion.columns]
tipos_dato = [str(df_educacion[col].dtype) for col in df_educacion.columns]
porcentaje_cardinalidad = [(df_educacion[col].nunique() / len(df_educacion) * 100) for col in df_educacion.columns]

cardinalidad = pd.DataFrame({
    'Columna': df_educacion.columns,
    'Valores_Únicos': valores_unicos,
    'Tipo_Dato': tipos_dato,
    '% Cardinalidad': [round(p, 2) for p in porcentaje_cardinalidad]
}).sort_values(by='Valores_Únicos', ascending=False)

display(cardinalidad)

 Distribución de Variables Categóricas - Top 15

In [ ]:
print("Distribución de Variables Categóricas (Top 15 valores):")
categorical_cols = df_educacion.select_dtypes(include=['object']).columns

for col in categorical_cols:
    print(f"\n{'='*80}")
    print(f"Columna: {col}")
    print(f"{'='*80}")
    value_counts = df_educacion[col].value_counts()
    print(f"Valores únicos: {df_educacion[col].nunique():,}")
    print(f"\nTop 15 valores más frecuentes:")
    display(pd.DataFrame({
        'Valor': value_counts.head(15).index,
        'Frecuencia': value_counts.head(15).values,
        'Porcentaje': (value_counts.head(15).values / len(df_educacion) * 100).round(2)
    }))

Análisis de Institución Educativa

In [ ]:
# Buscar columna de institución
inst_cols = [col for col in df_educacion.columns if 'INST' in col.upper() or 'CENTRO' in col.upper() or 'UNIVERSIDAD' in col.upper()]

if inst_cols:
    col_inst = inst_cols[0]
    print(f" Análisis de {col_inst}:")
    print("=" * 80)

    inst_counts = df_educacion[col_inst].value_counts()

    print(f"\nTotal de instituciones únicas: {df_educacion[col_inst].nunique():,}")
    print(f"\nTop 20 instituciones educativas:")

    display(pd.DataFrame({
        'Institución': inst_counts.head(20).index,
        'Cantidad': inst_counts.head(20).values,
        'Porcentaje': (inst_counts.head(20).values / len(df_educacion) * 100).round(2)
    }))

    # Visualización
    plt.figure(figsize=(14, 8))
    top_inst = inst_counts.head(15)
    plt.barh(range(len(top_inst)), top_inst.values)
    plt.yticks(range(len(top_inst)), [str(x)[:40] for x in top_inst.index])
    plt.xlabel('Cantidad de Registros')
    plt.title('Top 15 Instituciones Educativas')
    plt.tight_layout()
    plt.show()
else:
    print("No se encontró columna de institución educativa")

Análisis de Nivel Educativo

In [ ]:
# Buscar columna de nivel educativo
nivel_cols = [col for col in df_educacion.columns if 'NIVEL' in col.upper() or 'GRADO' in col.upper() or 'CICLO' in col.upper()]

if nivel_cols:
    col_nivel = nivel_cols[0]
    print(f" Análisis de {col_nivel}:")
    print("=" * 80)

    nivel_counts = df_educacion[col_nivel].value_counts()

    print(f"\nNiveles educativos únicos: {df_educacion[col_nivel].nunique()}")

    display(pd.DataFrame({
        'Nivel': nivel_counts.index,
        'Cantidad': nivel_counts.values,
        'Porcentaje': (nivel_counts.values / len(df_educacion) * 100).round(2)
    }))

    # Visualización - Pie chart
    plt.figure(figsize=(10, 8))
    if len(nivel_counts) <= 10:
        plt.pie(nivel_counts.values, labels=[str(x)[:30] for x in nivel_counts.index], autopct='%1.1f%%', startangle=90)
        plt.title(f'Distribución de {col_nivel}')
        plt.axis('equal')
    else:
        # Bar chart si hay muchos niveles
        plt.bar(range(len(nivel_counts)), nivel_counts.values)
        plt.xticks(range(len(nivel_counts)), [str(x)[:20] for x in nivel_counts.index], rotation=45, ha='right')
        plt.ylabel('Cantidad')
        plt.title(f'Distribución de {col_nivel}')
    plt.tight_layout()
    plt.show()
else:
    print(" No se encontró columna de nivel educativo")

Análisis de Carrera/Especialidad

In [ ]:
# Buscar columna de carrera
carrera_cols = [col for col in df_educacion.columns if 'CARRERA' in col.upper() or 'ESPECIALIDAD' in col.upper() or 'PROFESION' in col.upper()]

if carrera_cols:
    col_carrera = carrera_cols[0]
    print(f" Análisis de {col_carrera}:")
    print("=" * 80)

    carrera_counts = df_educacion[col_carrera].value_counts()

    print(f"\nCarreras/Especialidades únicas: {df_educacion[col_carrera].nunique():,}")
    print(f"\nTop 25 carreras más frecuentes:")

    display(pd.DataFrame({
        'Carrera': carrera_counts.head(25).index,
        'Cantidad': carrera_counts.head(25).values,
        'Porcentaje': (carrera_counts.head(25).values / len(df_educacion) * 100).round(2)
    }))

    # Visualización
    plt.figure(figsize=(14, 10))
    top_carreras = carrera_counts.head(20)
    plt.barh(range(len(top_carreras)), top_carreras.values)
    plt.yticks(range(len(top_carreras)), [str(x)[:50] for x in top_carreras.index])
    plt.xlabel('Cantidad de Registros')
    plt.title('Top 20 Carreras/Especialidades')
    plt.tight_layout()
    plt.show()
else:
    print(" No se encontró columna de carrera/especialidad")

 Análisis de Fechas

In [ ]:
# Buscar columnas de fecha
fecha_cols = [col for col in df_educacion.columns if 'FECHA' in col.upper() or 'DATE' in col.upper() or 'INICIO' in col.upper() or 'FIN' in col.upper()]

if fecha_cols:
    print(" Análisis de Fechas:")
    print("=" * 80)

    for col_fecha in fecha_cols:
        print(f"\n{col_fecha}:")
        print(f"  - Valores no nulos: {df_educacion[col_fecha].notna().sum():,}")
        print(f"  - Valores nulos: {df_educacion[col_fecha].isna().sum():,}")

        # Intentar convertir a datetime
        try:
            df_temp = df_educacion[col_fecha].dropna()
            if len(df_temp) > 0:
                # Mostrar ejemplos de valores
                print(f"  - Ejemplos de valores: {df_temp.head(5).tolist()}")
        except Exception as e:
            print(f"  - Error al analizar: {str(e)[:50]}")
else:
    print(" No se encontraron columnas de fecha")

Análisis de Postulantes Únicos

In [ ]:
# Buscar columna de ID de postulante
id_post_cols = [col for col in df_educacion.columns if 'POST' in col.upper() and 'ID' in col.upper()]

if id_post_cols:
    col_id_post = id_post_cols[0]
    print(f" Análisis de {col_id_post}:")
    print("=" * 80)

    postulantes_unicos = df_educacion[col_id_post].nunique()
    total_registros = len(df_educacion)

    print(f"Total de postulantes únicos: {postulantes_unicos:,}")
    print(f"Total de registros: {total_registros:,}")
    print(f"Promedio de registros educativos por postulante: {total_registros/postulantes_unicos:.2f}")

    # Distribución de registros educativos por postulante
    educacion_por_persona = df_educacion[col_id_post].value_counts()

    print(f"\n Distribución de registros educativos por postulante:")
    dist_educ = educacion_por_persona.value_counts().sort_index()

    # Mostrar solo primeros valores si hay muchos
    display(pd.DataFrame({
        'Num_Registros_Educativos': dist_educ.head(20).index,
        'Cantidad_Postulantes': dist_educ.head(20).values,
        'Porcentaje': (dist_educ.head(20).values / postulantes_unicos * 100).round(2)
    }))

    # Visualización
    plt.figure(figsize=(12, 6))
    top_dist = dist_educ.head(15)
    plt.bar(top_dist.index.astype(str), top_dist.values)
    plt.xlabel('Número de Registros Educativos')
    plt.ylabel('Cantidad de Postulantes')
    plt.title('Distribución de Número de Registros Educativos por Postulante (Top 15)')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print(" No se encontró columna de ID de postulante")

Análisis de Calidad de Datos

In [ ]:
print(" Análisis de Calidad de Datos:")
print("=" * 80)

# Valores vacíos o espacios en blanco
print("\n Análisis de valores vacíos/espacios:")
for col in df_educacion.select_dtypes(include=['object']).columns:
    empty_count = (df_educacion[col].str.strip() == '').sum() if df_educacion[col].dtype == 'object' else 0
    if empty_count > 0:
        print(f"   {col}: {empty_count:,} valores vacíos ({empty_count/len(df_educacion)*100:.2f}%)")

# Valores únicos muy bajos (posible problema de calidad)
print("\n Columnas con baja variabilidad:")
for col in df_educacion.columns:
    unique_ratio = df_educacion[col].nunique() / len(df_educacion)
    if unique_ratio < 0.01 and df_educacion[col].nunique() > 1:
        print(f"   {col}: {df_educacion[col].nunique()} valores únicos ({unique_ratio*100:.2f}%)")

# Longitud de strings
print("\n Análisis de longitud de texto:")
for col in df_educacion.select_dtypes(include=['object']).columns[:5]:  # Primeras 5 columnas
    if df_educacion[col].dtype == 'object':
        longitudes = df_educacion[col].dropna().astype(str).str.len()
        print(f"   {col}:")
        print(f"      - Longitud mínima: {longitudes.min()}")
        print(f"      - Longitud máxima: {longitudes.max()}")
        print(f"      - Longitud promedio: {longitudes.mean():.1f}")